## Logistic regression: Bankruptcy prediction

The focus of this project is the construction of logistic regression model, which will classify bank clients bankruptcy based on their background.

The sample data - 700 client profiles with information on their:
- Age in years
- Level of education
- Years with current employer
- Years at current address
- Household income in thousands
- Debt to income ratio (x100)
- Credit card debt in thousands
- Other debt in thousands

In [1]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm

import sklearn.metrics as sklm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression as LR

## 1. Read the data

In [2]:
# read the training and test samples
df_bank_train=pd.read_csv("bankloans_train.csv")
df_bank_test=pd.read_csv("bankloans_test.csv")

df_bank_train.head()

,default,age,educ,employ,address,income,debtinc,creddebt,othdebt
0,0,38,1,11,13,35,8.0,1.27,1.53
1,0,44,2,18,4,74,13.4,4.52,5.39
2,0,33,1,6,14,21,7.5,0.57,1.01
3,0,25,2,6,2,26,4.5,0.07,1.10
4,0,26,2,8,1,40,11.8,0.44,4.28


In [3]:
# get X_train and y_train
X_train = df_bank_train.drop("default", axis=1)
X_train.head()

,age,educ,employ,address,income,debtinc,creddebt,othdebt
0,38,1,11,13,35,8.0,1.27,1.53
1,44,2,18,4,74,13.4,4.52,5.39
2,33,1,6,14,21,7.5,0.57,1.01
3,25,2,6,2,26,4.5,0.07,1.10
4,26,2,8,1,40,11.8,0.44,4.28


In [4]:
# get y_train
y_train = df_bank_train["default"]
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: default, dtype: int64

In [5]:
# get X_test and y_test
X_test = df_bank_test.drop("default", axis=1)
y_test = df_bank_test["default"]

## 2. Run a traditional logistic regression

In [6]:
# estimate a logistic regression (with SM)
XX_train = sm.add_constant(X_train)
sm_logit_model = sm.Logit(y_train, XX_train)
sm_logit_model.fit().summary()

Optimization terminated successfully.
         Current function value: 0.384243
         Iterations 7


C:\Users\danil\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                  560
Model:                          Logit   Df Residuals:                      551
Method:                           MLE   Df Model:                            8
Date:                Sun, 15 Dec 2019   Pseudo R-squ.:                  0.3191
Time:                        02:18:55   Log-Likelihood:                -215.18
converged:                       True   LL-Null:                       -316.00
Covariance Type:            nonrobust   LLR p-value:                 2.867e-39
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6920      0.714     -2.371      0.018      -3.091      -0.293
age            0.0305      0.019      1.602      0.109      -0.007       0.068
educ           0.1524      0.137      1.114      0.265      -0.116       0.421
employ        -0.2620      0.037     -7.022      0.000      -0.335      -0.189
address       -0.0987      0.026     -3.835      0.000      -0.149      -0.048
income        -0.0053      0.010     -0.508      0.611      -0.026       0.015
debtinc        0.0558      0.037      1.497      0.134      -0.017       0.129
creddebt       0.6147      0.127      4.852      0.000       0.366       0.863
othdebt        0.1045      0.096      1.094      0.274      -0.083       0.292
==============================================================================
"""

## 3. Redundant feature elimination

In [7]:
# set up the the LR-estimator with SKLearn
sk_logreg = LR(random_state=0, solver='lbfgs', max_iter = 1000)

rfe = RFE(sk_logreg, n_features_to_select=None)
rfe = rfe.fit(X_train, y_train.values.ravel())

# print out the results of RFE
print(rfe.support_)
print(rfe.ranking_)

[False False  True  True False False  True  True]
[4 2 1 1 5 3 1 1]


## 4. Making predictions

In [8]:
# estimate an SKlearn logistic regression
sk_clfr = sk_logreg.fit(X_train, y_train)

### 4.1. Prediction of classes

In [9]:
# predict for the test sample (with the default cut-off value 50%)
y_test_pred = sk_clfr.predict(X_test)
y_test_pred

array([0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [10]:
# construct a confusion matrix
CM50 = pd.crosstab(index=y_test_pred, columns=y_test, margins=False)
print(CM50)

default   0   1
row_0          
0        90  23
1         8  19


In [11]:
# classification report
print(sklm.classification_report(y_true=y_test, y_pred = y_test_pred))

              precision    recall  f1-score   support

           0       0.80      0.92      0.85        98
           1       0.70      0.45      0.55        42

    accuracy                           0.78       140
   macro avg       0.75      0.69      0.70       140
weighted avg       0.77      0.78      0.76       140



In [12]:
# get the accuracy directly 
sk_clfr.score(X_test, y_test)

0.7785714285714286

### 4.2. Prediction of probabilities

In [13]:
y_test_pred_probs = sk_clfr.predict_proba(X_test)
y_test_pred_probs[:4]

array([[0.86496633, 0.13503367],
       [0.98678785, 0.01321215],
       [0.73687035, 0.26312965],
       [0.98566757, 0.01433243]])

In [14]:
# change cut-off value from 50% to 25%
y_test_pred_classes = np.array([(y_test_pred_probs[i][1]>0.25).astype(int) for i in range(len(X_test))])
y_test_pred_classes[:4]

array([0, 0, 1, 0])

In [15]:
# construct confusion matrix again
CM25 = pd.crosstab(index=y_test_pred_classes, columns=y_test, margins=False)
print(CM25)

default   0   1
row_0          
0        68  11
1        30  31


In [16]:
# classification report
print(sklm.classification_report(y_true=y_test, y_pred = y_test_pred_classes))

              precision    recall  f1-score   support

           0       0.86      0.69      0.77        98
           1       0.51      0.74      0.60        42

    accuracy                           0.71       140
   macro avg       0.68      0.72      0.69       140
weighted avg       0.75      0.71      0.72       140

